In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_distribution

# Create a new qubit called 'Q'
q = QuantumRegister(2, "Q")

# Create a new circuit with two qubits
qc = QuantumCircuit(q)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Apply a CNOT gate with qubit 0 as control and qubit 1 as target
qc.cx(0, 1)

qc.measure_all()

# Return a drawing of the circuit using MatPlotLib ("mpl"). This is the
# last line of the cell, so the drawing appears in the cell output.
# Remove the "mpl" argument to get a text drawing.
qc.draw(output="mpl")

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import Aer
import os

KEY = os.environ["IBM_API_TOKEN"]

QiskitRuntimeService.save_account(channel="ibm_quantum",
								  token=KEY, overwrite=True,
								  set_as_default=True)

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Optimize for quantum execution
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)

In [ ]:
target = backend.target
pass_manager = generate_preset_pass_manager(target=target, optimization_level=2, backend=backend)

# Transpile it by calling the run method of the pass manager
transpiled = pass_manager.run(qc)
 
# Draw it, excluding idle qubits from the diagram
transpiled.draw("mpl", idle_wires=False, style="iqp")

In [ ]:
# Imports from Qiskit Runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler, Options

# Here we can configure the sample, such as:
# sampler.options.default_shots = 500
sampler = Sampler(mode=backend)

# Turn on dynamical decoupling
sampler.options.dynamical_decoupling.enable = True
# Turn on gate twirling. Requires qiskit_ibm_runtime 0.23.0 or later.
sampler.options.twirling.enable_gates = True

# Create a job to be executed locally or remote
job = sampler.run([transpiled], shots=500)
job_id = job.job_id()
print(job_id)

In [ ]:
# Create histogram from job result
#job = service.job('ctbbagtwwtng0083dneg')
result = job.result()
dist = result[0].data.meas.get_counts()
plot_distribution(dist)

In [ ]:
plot_histogram(dist)